In [18]:
!pip install -U -q PyDrive
# !rm -rf Epoch*

# !pip install tensorflow==2.0.0-alpha0
!pip install -q tf-nightly-2.0-preview

     |████████████████████████████████| 87.6MB 1.3MB/s 
     |████████████████████████████████| 430kB 46.0MB/s 
ERROR: thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.


In [19]:
import os
import time
import pickle
import random 
import itertools

import pandas as pd
import numpy as np
import tensorflow as tf

from skimage.io import imread#,imshow
from skimage.filters import threshold_otsu
from skimage.morphology import convex_hull_image
from skimage.exposure import equalize_adapthist
from skimage.transform import resize

from matplotlib import pyplot as plt

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

tf.test.gpu_device_name()

%load_ext tensorboard

The tensorboard module is not an IPython extension.


In [20]:
tf.__version__

'2.0.0-alpha0'

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
%%capture
Dir = os.listdir("./")

if ("MC_Finger_Vein_Database" not in Dir):
    if ("Finger Vein Database.rar" not in Dir):
        checkpoint = drive.CreateFile({'id': '1Qq2TbwoolE9BGOD8Ooqon8KK1c_AkDhs'})
        checkpoint.GetContentFile('Finger_Vein_Database.zip')
#         !wget http://mla.sdu.edu.cn/sjk/Finger%20Vein%20Database.rar
        del checkpoint
    !unzip "Finger_Vein_Database.zip"
    !mv "Finger" "MC_Finger_Vein_Database"
del Dir
# if ("Finger_Vein_Database" not in Dir):
#     if ("Finger Vein Database.rar" not in Dir):
#         !wget http://mla.sdu.edu.cn/sjk/Finger%20Vein%20Database.rar
#     !unrar x -r "Finger Vein Database.rar"
#     !mv "Finger Vein Database" "Finger_Vein_Database"
# # if ('ngrok-stable-linux-amd64.zip' not in Dir):
# #   !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# #   !unzip ngrok-stable-linux-amd64.zip
  
# del Dir
# !cp -a '/content/gdrive/My Drive/Deepnet/.' ./ 

In [0]:
DATADIR = "MC_Finger_Vein_Database"
learning_rate = 0.001
num_epochs = 50
# global_step=0
# tf.train.exponential_decay(
#     0.01,
#     global_step,
#     100000, 
#     0.96.
#     staircase=True,
# )

In [0]:
def load_image(addr):
    image = imread(addr,flatten=True)
#     thresh = threshold_otsu(image)
#     binary = image <= thresh
#     image = image * convex_hull_image(binary == 0)
#     image = equalize_adapthist(image)
    return resize(image, (image.shape[0], image.shape[1],1) )

In [0]:
def database():
    traning_data=[]
    label = []
    n=len(os.listdir(DATADIR))
    remain=1
    for person in os.listdir(DATADIR):
        path=os.path.join(DATADIR,person)
        print("Getting data at "+path+"----Remaining Files : "+str(n-remain))
#         if((n-remain)<95):
#             break
        remain+=1
    
        for hand in os.listdir(path):
            path_=os.path.join(path,hand)
        temp=[]
        previous_finger=None
        for finger in os.listdir(path_):
            img_path = os.path.join(path_,finger)
            tag = person+"_"+hand+"_"+finger[:-6]
            traning_data.append(load_image(img_path))
            label.append(tag)
    return np.array(traning_data),np.array(label)

In [0]:
def get_neg_duplet(features):
    df = pd.DataFrame(features)
    duplet = []
    for index, row in df.iterrows():
        same_tag = df.loc[df.iloc[:, -1] == row.iloc[-1]]
        same_tag_indexes = list(set(same_tag.index) - {index})
        diff_tag_indexes = list(set(df.index) - set(same_tag_indexes) - {index})
        anchor = row[0]
        neg = df.iloc[random.choice(diff_tag_indexes), :].iloc[0]
        duplet.append({"anchor":anchor, "second":neg, "label":0})
    
    return np.array(duplet)

def get_pos_duplet(features):
    df = pd.DataFrame(features)
    duplet = []
    label = []
    for index, row in df.iterrows():
        same_tag = df.loc[df.iloc[:, -1] == row.iloc[-1]]
        if (same_tag[1].iloc[0] not in label):
            label.append(same_tag[1].iloc[0])
            same_tag_indexes = list(set(same_tag.index) - {index})
            anchor = row[0]
            for indexx in same_tag_indexes:
                pos = df.iloc[indexx, :].iloc[0]
                duplet.append({"anchor":anchor, "second":pos, "label":1})
    
    return np.array(duplet)


In [0]:
def create_embedding_network(input_shape):
    input_ = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Conv2D(64, (7, 7), strides=(2, 2), padding = "valid", name="conv1", activation="relu" , input_shape= input_shape[1:])(input_)
    x = tf.keras.layers.MaxPooling2D(pool_size= (3, 3), strides = (2, 2), padding = "valid", name="pool1")(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv2D(64, (1, 1), strides=(1, 1), padding = "valid", name="conv2a", activation="relu")(x)
    x = tf.keras.layers.Conv2D(192, (3, 3), strides=(1, 1), padding = "valid", name="conv2", activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size= (3, 3), strides = (2, 2), padding = "valid", name="pool2")(x)


    x= tf.keras.layers.Conv2D(192, (1, 1), strides=(1, 1), padding = "valid", name="conv3a", activation="relu")(x)
    x= tf.keras.layers.Conv2D(384, (3, 3), strides=(1, 1), padding = "valid", name="conv3", activation="relu")(x)
    x= tf.keras.layers.MaxPooling2D(pool_size= (3, 3), strides = (2, 2), padding = "valid", name="pool3")(x)

    x= tf.keras.layers.Conv2D(384, (1, 1), strides=(1, 1), padding = "valid", name="conv4a", activation="relu")(x)
    x= tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding = "valid", name="conv4", activation="relu")(x)

    x= tf.keras.layers.Conv2D(256, (1, 1), strides=(1, 1), padding = "valid", name="conv5a", activation="relu")(x)
    x= tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding = "valid", name="conv5", activation="relu")(x)

    x= tf.keras.layers.Conv2D(256, (1, 1), strides=(1, 1), padding = "valid", name="conv6a", activation="relu")(x)
    x= tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding = "valid", name="conv6", activation="relu")(x)
    x= tf.keras.layers.MaxPooling2D(pool_size= (3, 3), strides = (2, 2), padding = "valid", name="pool4")(x)
    #   x= tf.keras.layers.Conv2D(1024, (1, 1), strides=(1, 1), padding = "valid", name="conva5b", activation="relu")(x)
    x = tf.keras.layers.Flatten()(x)
#   x = tf.keras.layers.Dense(512, activation="sigmoid")(x)
    model = tf.keras.models.Model(inputs=input_, outputs=x)
    return model

In [0]:
def build_model(input_shape):
    global learning_rate
    anchor = tf.keras.layers.Input(shape=input_shape)
    positive = tf.keras.layers.Input(shape=input_shape)

    embedding_network = create_embedding_network(input_shape)

    print(embedding_network.summary())

    anchor_embedding = embedding_network(anchor)
    positive_embedding = embedding_network(positive)
    x = tf.keras.layers.Concatenate()([anchor_embedding, positive_embedding])

    x = tf.keras.layers.Dense(2048, activation="sigmoid")(x)
    prediction = tf.keras.layers.Dense(1,activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=[anchor,positive], outputs=prediction)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [0]:
class SaveModel(tf.keras.callbacks.Callback):
#   global drive
    def on_train_begin(self, logs={}):
        self.epoch_counter=1

    def on_epoch_end(self, batch, logs={}):
        try:
            if(self.epoch_counter%80==0):
                auth.authenticate_user()
                gauth = GoogleAuth()
                gauth.credentials = GoogleCredentials.get_application_default()
                drive = GoogleDrive(gauth)

                file_name = "Epoch_"+str(self.epoch_counter)+".h5"
                self.model.save_weights(file_name)
                model_file = drive.CreateFile({'title' : file_name, "parents": 
                                               [{"kind": "drive#childList","id": "11el4YYRbi7xpoNRbvbgE0LDo47ygBazP"}]})
                model_file.SetContentFile(file_name)
                model_file.Upload()
        except Exception as e:
            print("Model Not Saved on drive")
        self.epoch_counter+=1

In [28]:
training_data,label = database()

Getting data at MC_Finger_Vein_Database/082----Remaining Files : 105
Getting data at MC_Finger_Vein_Database/091----Remaining Files : 104


/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:54: UserWarning: `flatten` has been deprecated in favor of `as_gray` and will be removed in v0.16.
  warn('`flatten` has been deprecated in favor of `as_gray`'
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Getting data at MC_Finger_Vein_Database/092----Remaining Files : 103
Getting data at MC_Finger_Vein_Database/015----Remaining Files : 102
Getting data at MC_Finger_Vein_Database/007----Remaining Files : 101
Getting data at MC_Finger_Vein_Database/033----Remaining Files : 100
Getting data at MC_Finger_Vein_Database/010----Remaining Files : 99
Getting data at MC_Finger_Vein_Database/104----Remaining Files : 98
Getting data at MC_Finger_Vein_Database/095----Remaining Files : 97
Getting data at MC_Finger_Vein_Database/077----Remaining Files : 96
Getting data at MC_Finger_Vein_Database/001----Remaining Files : 95
Getting data at MC_Finger_Vein_Database/087----Remaining Files : 94
Getting data at MC_Finger_Vein_Database/078----Remaining Files : 93
Getting data at MC_Finger_Vein_Database/049----Remaining Files : 92
Getting data at MC_Finger_Vein_Database/003----Remaining Files : 91
Getting data at MC_Finger_Vein_Database/017----Remaining Files : 90
Getting data at MC_Finger_Vein_Database/025-

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((training_data, label))
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [37]:
batch_size=64
dataset = dataset.shuffle(3000).repeat().batch(batch_size).prefetch(AUTOTUNE)

# data = get_neg_duplet(training_data)
# data2 = get_pos_duplet(training_data)
# data = np.concatenate((data, data2), axis=None)
# np.random.shuffle(data)
# test_data = data[:int(data.shape[0]*20/100)]
# data = data[int(data.shape[0]*20/100):]

NameError: ignored

<BatchDataset shapes: ((None, 240, 320, 1), (None,)), types: (tf.float64, tf.string)>

In [0]:
# !mv 'Epoch20.h5' 'WithGaborEpoch20.h5'
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# file_name = "Epoch_"+str(20)+".h5"
# model.save_weights(file_name)
# model_file = drive.CreateFile({'title' : file_name, "parents": 
#                                [{"kind": "drive#childList","id": "11el4YYRbi7xpoNRbvbgE0LDo47ygBazP"}]})
# model_file.SetContentFile(file_name)
# model_file.Upload()

In [16]:
model = build_model(training_data[0][0].shape)
model.summary()
# model.load_weights("Epoch_full_100.h5")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 240, 320, 1)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 117, 157, 64)      3200      
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 58, 78, 64)        0         
_________________________________________________________________
batch_normalization_v2 (Batc (None, 58, 78, 64)        256       
_________________________________________________________________
conv2a (Conv2D)              (None, 58, 78, 64)        4160      
_________________________________________________________________
conv2 (Conv2D)               (None, 56, 76, 192)       110784    
_________________________________________________________________
batch_normalization_v2_1 (Ba (None, 56, 76, 192)       768   

In [17]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


In [14]:
# smCallBack=SaveModel()


model.fit([np.array([d["anchor"] for d in data]),np.array([d["second"] for d in data])], 
            y= np.array([d["label"] for d in data]) , epochs=40)

  

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 240, 320, 1)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 117, 157, 64)      3200      
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 58, 78, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 58, 78, 64)        256       
_________________________________________________________________
conv2a (Conv2D)              (None, 58, 78, 64)        4160      
_________________________________________________________________
conv2 (Conv2D)               (None, 56, 76, 192)       110784    
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 76, 192)       768   

AttributeError: ignored

In [23]:
k =model.evaluate([np.array([d["anchor"] for d in test_data]),np.array([d["second"] for d in test_data])], 
          y= np.array([d["label"] for d in test_data]))
print(k)

699/699 [==============================] - 3s 5ms/sample - loss: 0.6909 - acc: 0.5365
[0.6909157120448155, 0.53648067]


In [0]:
model.evaluate([np.array([d["anchor"] for d in data]),np.array([d["second"] for d in data])], 
          y= np.array([d["label"] for d in data]))


5597/5597 [==============================] - 24s 4ms/sample - loss: 0.1042 - acc: 0.9691


[0.104211958975521, 0.9690906]